In [1]:
from __future__ import annotations

import os
import numpy as np
import pandas as pd
from collections import defaultdict, deque
from typing import Dict, List, Tuple



In [2]:

# =========================
# Settings
# =========================
DATA_DIR = "synthetic_data"
N_ITER = 10_000
SEED = 123
OUT_DIR = "mc_outputs"

rng = np.random.default_rng(SEED)


In [3]:
# =========================
# Graph utilities
# =========================
def topological_order(task_codes: List[str], preds_map: Dict[str, List[str]]) -> List[str]:
    indeg = {t: 0 for t in task_codes}
    graph = defaultdict(list)
    for t in task_codes:
        for p in preds_map.get(t, []):
            if p not in indeg:
                continue
            graph[p].append(t)
            indeg[t] += 1
    q = deque([t for t in task_codes if indeg[t] == 0])
    order = []
    while q:
        u = q.popleft()
        order.append(u)
        for v in graph[u]:
            indeg[v] -= 1
            if indeg[v] == 0:
                q.append(v)
    if len(order) != len(task_codes):
        raise ValueError("Cycle detected or missing nodes in DAG.")
    return order


def critical_path_duration(order: List[str], preds_map: Dict[str, List[str]], dur: Dict[str, float]) -> float:
    ef = {}
    for t in order:
        preds = preds_map.get(t, [])
        start = max([ef[p] for p in preds], default=0.0)
        ef[t] = start + float(dur[t])
    return max(ef.values()) if ef else 0.0


In [4]:
# =========================
# Risk application
# =========================
def apply_risks_once(
    base_dur: Dict[str, float],
    risks: pd.DataFrame,
    rng: np.random.Generator
) -> Tuple[Dict[str, float], float]:
    """
    Returns adjusted durations and extra lump cost for ONE iteration.
    Risk types:
      - ADD: add triangular days to target tasks
      - MUL: multiply target tasks by lognormal, AND also add triangular days (for late testing drag)
    """
    dur = dict(base_dur)
    extra_cost = 0.0

    for _, rrow in risks.iterrows():
        p = float(rrow["probability"])
        if rng.random() >= p:
            continue  # risk did not occur

        targets = str(rrow["targets"]).split(";") if pd.notna(rrow["targets"]) else []
        targets = [t for t in targets if t in dur]

        # schedule additive piece (always available)
        add_o = float(rrow["sched_add_tri_o"])
        add_m = float(rrow["sched_add_tri_m"])
        add_p = float(rrow["sched_add_tri_p"])
        add_days = float(rng.triangular(add_o, add_m, add_p))

        # cost lump sum
        c_o = float(rrow["cost_lump_tri_o"])
        c_m = float(rrow["cost_lump_tri_m"])
        c_p = float(rrow["cost_lump_tri_p"])
        extra_cost += float(rng.triangular(c_o, c_m, c_p))

        if rrow["risk_type"] == "ADD":
            for t in targets:
                dur[t] += add_days

        elif rrow["risk_type"] == "MUL":
            mu = float(rrow["mul_logn_mu"])
            sigma = float(rrow["mul_logn_sigma"])
            factor = float(rng.lognormal(mu, sigma))
            for t in targets:
                dur[t] *= factor
                dur[t] += add_days  # extra drag on top

        else:
            raise ValueError(f"Unknown risk_type: {rrow['risk_type']}")

    return dur, extra_cost



In [5]:
# =========================
# Main run
# =========================
def run():
    os.makedirs(OUT_DIR, exist_ok=True)

    projects = pd.read_csv(os.path.join(DATA_DIR, "projects.csv"))
    tasks = pd.read_csv(os.path.join(DATA_DIR, "tasks.csv"))
    risks = pd.read_csv(os.path.join(DATA_DIR, "risks.csv"))

    summary_rows = []
    long_rows = []

    for pid, pinfo in projects.set_index("project_id").iterrows():
        tdf = tasks[tasks["project_id"] == pid].copy()
        rdf = risks[risks["project_id"] == pid].copy()

        # preds map
        preds_map = {}
        for _, row in tdf.iterrows():
            pred = str(row["pred"]).strip()
            preds_map[row["task_code"]] = [] if pred in ["", "nan", "None"] else pred.split(";")

        task_codes = list(tdf["task_code"].values)
        order = topological_order(task_codes, preds_map)

        # CPM baseline: use likely_m only
        cpm_dur_map = {row["task_code"]: float(row["likely_m"]) for _, row in tdf.iterrows()}
        cpm_duration = critical_path_duration(order, preds_map, cpm_dur_map)
        burn = float(pinfo["burn_rate_per_day"])
        fixed_cost = float(pinfo["fixed_cost"])
        cpm_cost = cpm_duration * burn + fixed_cost

        durations = np.zeros(N_ITER, dtype=float)
        costs = np.zeros(N_ITER, dtype=float)

        # Monte Carlo
        for i in range(N_ITER):
            base_dur = {}
            for _, row in tdf.iterrows():
                o = float(row["opt_o"])
                m = float(row["likely_m"])
                p = float(row["pess_p"])
                base_dur[row["task_code"]] = float(rng.triangular(o, m, p))

            adj_dur, extra_cost = apply_risks_once(base_dur, rdf, rng)

            proj_duration = critical_path_duration(order, preds_map, adj_dur)
            proj_cost = proj_duration * burn + fixed_cost + extra_cost

            durations[i] = proj_duration
            costs[i] = proj_cost

        # stats
        def pct(x, q): return float(np.quantile(x, q))
        d_mean = float(durations.mean())
        c_mean = float(costs.mean())

        d_p50, d_p80, d_p90 = pct(durations, 0.50), pct(durations, 0.80), pct(durations, 0.90)
        c_p50, c_p80, c_p90 = pct(costs, 0.50), pct(costs, 0.80), pct(costs, 0.90)

        # overrun probabilities vs CPM baseline
        p_cost_over = float((costs > cpm_cost).mean())
        p_sched_over = float((durations > cpm_duration).mean())

        summary_rows.append({
            "project_id": pid,
            "size_bucket": pinfo["size_bucket"],
            "risk_level_bucket": pinfo["risk_level_bucket"],
            "late_concentration": pinfo["late_concentration"],
            "tail_type": pinfo["tail_type"],
            "coupling": pinfo["coupling"],
            "n_tasks": int(pinfo["n_tasks"]),
            "n_streams": int(pinfo["n_streams"]),
            "burn_rate_per_day": burn,
            "fixed_cost": fixed_cost,

            "cpm_duration": cpm_duration,
            "cpm_cost": cpm_cost,

            "mc_duration_mean": d_mean,
            "mc_duration_p50": d_p50,
            "mc_duration_p80": d_p80,
            "mc_duration_p90": d_p90,

            "mc_cost_mean": c_mean,
            "mc_cost_p50": c_p50,
            "mc_cost_p80": c_p80,
            "mc_cost_p90": c_p90,

            "p_duration_over_cpm": p_sched_over,
            "p_cost_over_cpm": p_cost_over,
        })

        # optional long format (for ML later)
        # keep it lighter: store only percentiles per project OR store sampled points
        # Here: store sampled points but you can turn it off if file gets big.
        for i in range(N_ITER):
            long_rows.append({
                "project_id": pid,
                "duration": durations[i],
                "cost": costs[i],
                "cpm_duration": cpm_duration,
                "cpm_cost": cpm_cost,
            })

        print(f"{pid}: CPM {cpm_duration:.1f}d | MC P50 {d_p50:.1f}d P90 {d_p90:.1f}d | P(cost>CPM)={p_cost_over:.2f}")

    summary_df = pd.DataFrame(summary_rows).sort_values(["risk_level_bucket", "size_bucket", "project_id"])
    summary_path = os.path.join(OUT_DIR, "project_summary.csv")
    summary_df.to_csv(summary_path, index=False)

    long_df = pd.DataFrame(long_rows)
    long_path = os.path.join(OUT_DIR, "sim_long.csv")
    long_df.to_csv(long_path, index=False)

    print("\nSaved:")
    print(" -", summary_path)
    print(" -", long_path)
    print("\nSummary head:")
    print(summary_df.head(8))


if __name__ == "__main__":
    run()

P01: CPM 77.1d | MC P50 79.8d P90 111.1d | P(cost>CPM)=0.78
P02: CPM 101.7d | MC P50 110.8d P90 149.3d | P(cost>CPM)=0.87
P03: CPM 146.7d | MC P50 171.9d P90 223.7d | P(cost>CPM)=0.94
P04: CPM 75.3d | MC P50 111.6d P90 173.3d | P(cost>CPM)=0.97
P05: CPM 108.1d | MC P50 115.9d P90 148.8d | P(cost>CPM)=0.85
P06: CPM 133.1d | MC P50 143.4d P90 181.6d | P(cost>CPM)=0.84
P07: CPM 73.1d | MC P50 93.6d P90 142.3d | P(cost>CPM)=0.94
P08: CPM 94.4d | MC P50 131.0d P90 200.2d | P(cost>CPM)=0.97
P09: CPM 126.9d | MC P50 131.8d P90 149.1d | P(cost>CPM)=0.76
P10: CPM 74.1d | MC P50 83.8d P90 125.3d | P(cost>CPM)=0.88
P11: CPM 98.2d | MC P50 114.8d P90 163.7d | P(cost>CPM)=0.93
P12: CPM 118.1d | MC P50 156.0d P90 230.5d | P(cost>CPM)=0.97
P13: CPM 77.4d | MC P50 81.0d P90 113.4d | P(cost>CPM)=0.81
P14: CPM 86.2d | MC P50 93.5d P90 132.1d | P(cost>CPM)=0.81
P15: CPM 124.2d | MC P50 147.0d P90 194.9d | P(cost>CPM)=0.95
P16: CPM 82.8d | MC P50 117.2d P90 182.5d | P(cost>CPM)=0.95
P17: CPM 84.7d | MC P5